In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import plspm.config as c
from plspm.mode import Mode
from plspm.plspm import Plspm
from plspm.scale import Scale
from plspm.scheme import Scheme

source = pd.read_csv("file:../data/corporate-reputation.csv", index_col=0)

In [2]:
# Remove missing data
source[source == -99] = np.NaN

# Mean Replacement
replaced_source = source.fillna(source.mean())

![Structure](https://i.imgur.com/NWbhsWw.png)

In [3]:
structure = c.Structure()
structure.add_path(["COMP"], ["CUSA", "CUSL"])
structure.add_path(["LIKE"], ["CUSA", "CUSL"])
structure.add_path(["CUSA"], ["CUSL"])

config = c.Config(
    structure.path(),
    scaled=True,
    # default_scale=Scale.NUM # Use this to scale all variables as numeric
)
config.add_lv_with_columns_named("COMP", Mode.A, replaced_source, "comp")
config.add_lv_with_columns_named("LIKE", Mode.A, replaced_source, "like")
config.add_lv_with_columns_named("CUSA", Mode.A, replaced_source, "cusa")
config.add_lv_with_columns_named("CUSL", Mode.A, replaced_source, "cusl")

In [4]:
# Start training
calculator = Plspm(
    replaced_source,
    config,
    Scheme.PATH,
    tolerance=1e-7,
    iterations=300
)

# Result Target

Summary structural(endogenous) path:

| | CUSA | CUSL |
| - | - | - |
| $R^2$     | 0.295 | 0.562 |
| Adj $R^2$ | 0.290 | 0.558 |
| COMP      | 0.162 | 0.009 |
| LIKE      | 0.424 | 0.342 |
| CUSA      |       | 0.504 |

Construct reliability metrics:

| | alpha | rhoC | AVE | rhoA |
| - | - | - | - | - |
| COMP  | 0.776 | 0.865 | 0.681 | 0.832 |
| LIKE  | 0.831 | 0.899 | 0.747 | 0.836 |
| CUSA  | 1.000 | 1.000 | 1.000 | 1.000 |
| CUSL | 0.831 | 0.899 | 0.748 | 0.839 |

In [6]:
pd.set_option('display.expand_frame_repr', False)
print('===unidimensionality')
print(calculator.unidimensionality())
print('\n===inner_summary')
print(calculator.inner_summary())
print('\n===inner_model')
print(calculator.inner_model())
print('\n===outer_model')
print(calculator.outer_model())
print('\n===crossloadings')
print(calculator.crossloadings())
print('\n===effects')
print(calculator.effects())
print('\n===goodness_of_fit')
print(calculator.goodness_of_fit())
print('\n===path_coefficients')
print(calculator.path_coefficients())

===unidimensionality
     mode  mvs  cronbach_alpha  dillon_goldstein_rho   eig_1st   eig_2nd
LIKE    A  3.0        0.830987              0.898759  2.242330  0.407392
COMP    A  3.0        0.776028              0.870505  2.076027  0.572377
CUSA    A  1.0             NaN              1.000000  1.000000       NaN
CUSL    A  3.0        0.830988              0.899116  2.245436  0.493181

===inner_summary
            type  r_squared  r_squared_adj  block_communality  mean_redundancy       ave
COMP   Exogenous   0.000000       0.000000           0.679386         0.000000  0.679386
CUSA  Endogenous   0.288909       0.284739           1.000000         0.288909  1.000000
CUSL  Endogenous   0.562603       0.558743           0.746900         0.420208  0.746900
LIKE   Exogenous   0.000000       0.000000           0.746112         0.000000  0.746112

===inner_model
              from    to  estimate  std error          t         p>|t|
index                                                           

In [ ]:
bootstrap = Plspm(
    replaced_source,
    config,
    Scheme.PATH,
    tolerance=1e-7,
    iterations=300,
    bootstrap=True,
    bootstrap_iterations=1000,
)

In [12]:
print('=== bootstrap paths ===')
print(bootstrap.bootstrap().paths())
print('\n=== bootstrap loading ===')
print(bootstrap.bootstrap().loading())
print('\n=== bootstrap r_squared ===')
print(bootstrap.bootstrap().r_squared())
print('\n=== bootstrap total_effects ===')
print(bootstrap.bootstrap().total_effects())
print('\n=== bootstrap weights ===')
print(bootstrap.bootstrap().weights())

=== bootstrap paths ===
              original      mean  std.error  perc.025  perc.975    t stat.
LIKE -> CUSA  0.408497  0.406224   0.060017  0.278297  0.526596   6.806364
LIKE -> CUSL  0.335506  0.336910   0.054991  0.228976  0.447891   6.101096
COMP -> CUSA  0.174906  0.178713   0.065384  0.053422  0.306277   2.675069
COMP -> CUSL  0.006355  0.008609   0.052909 -0.100248  0.108803   0.120107
CUSA -> CUSL  0.514525  0.513200   0.042740  0.428746  0.594721  12.038362

=== bootstrap loading ===
        original      mean     std.error  perc.025  perc.975       t stat.
comp_1  0.861824  0.862656  2.082086e-02  0.821515  0.902575  4.139235e+01
comp_2  0.787843  0.785618  3.198184e-02  0.714990  0.838591  2.463407e+01
comp_3  0.821415  0.818695  3.111982e-02  0.748834  0.866413  2.639523e+01
like_1  0.843785  0.843624  2.221050e-02  0.795565  0.881887  3.799037e+01
like_2  0.883252  0.883316  1.764350e-02  0.845665  0.915182  5.006106e+01
like_3  0.863845  0.863633  1.855840e-02  0.82530